In [1]:
import requests
import json
import time
from datetime import datetime
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import warnings
warnings.simplefilter('ignore', InsecureRequestWarning)
from bs4 import BeautifulSoup


request_id = "b6f656ec-6231-44af-961c-40dab68ea180"
# range_wanted = ("16:30", "17:30") # TODO système d'alerting
delay_between_tries = 60


# On peut récupérer une clé en cours de validité sur la page d'accueil de sncf-connect. Je pense que cette clé doit changer régulièrement ?
def get_api_key():
    headers = {
        "Host": "www.sncf-connect.com",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "fr,fr-FR;q=0.8,en-US;q=0.5,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
    }

    response = requests.get("https://www.sncf-connect.com", verify=False, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        script_tag = soup.find('script', {'type': 'application/json'})
        data = json.loads(script_tag.string)
        x_bff_key = data['bffServer']['key_value']
        return x_bff_key

    else:
        print(f"Erreur : {response.status_code}")
        raise Exception("Erreur dans la récupération de la clé API")

# attention, boucle infinie, y'a aucune condition d'arrêt là dedans pour l'instant
while(1):
    print(f"## Tentative de {datetime.now()}")
    print()
    
    headers = {
        "Host": "www.sncf-connect.com",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "fr,fr-FR;q=0.8,en-US;q=0.5,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br",
        "x-bff-key": get_api_key(),
        "Connection": "keep-alive",
        "Referer": "https://www.sncf-connect.com/app/home/shop/results/outward",
    }
    
    # on récupère les trajets proposés et le statut de chaque trajet
    response = requests.get(f"https://www.sncf-connect.com/bff/api/v1/itineraries/{request_id}?outward=true&plan=false", verify=False, headers=headers)

    if response.status_code == 200:
        results = response.json()
        proposals = results['output']['longDistance']['proposals']['proposals']
        for proposal in proposals:
            print(f"{proposal['departure']['timeLabel']} : {proposal['status']['isBookable']}")
        print()
    else:
        # Printing an error message if the request failed
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        
    time.sleep(delay_between_tries)

## Tentative de 2024-03-03 18:16:09.757115

05:33 : True
06:35 : True
07:00 : True
07:00 : True
07:35 : True
07:35 : True

## Tentative de 2024-03-03 18:17:11.493219

05:33 : True
06:35 : True
07:00 : True
07:00 : True
07:35 : True
07:35 : True

## Tentative de 2024-03-03 18:18:13.044140

05:33 : True
06:35 : True
07:00 : True
07:00 : True
07:35 : True
07:35 : True

## Tentative de 2024-03-03 18:19:14.818777

05:33 : True
06:35 : True
07:00 : True
07:00 : True
07:35 : True
07:35 : True



KeyboardInterrupt: 